# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [41]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [42]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vardo,70.3705,31.1107,0.99,86,75,8.75,NO,1675177596
1,1,bluff,-46.6000,168.3333,15.43,97,100,2.59,NZ,1675177569
2,2,hobyo,5.3505,48.5268,25.26,71,2,9.23,SO,1675177569
3,3,lumsden,50.6501,-104.8678,-17.77,95,100,1.23,CA,1675177570
4,4,guerrero negro,27.9769,-114.0611,12.88,80,8,3.35,MX,1675177380


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [43]:
#%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City',
    alpha = .5   
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [44]:
# Narrow down cities that fit criteria and drop any results with null values
clean_city_data = city_data_df.dropna()

# Drop any rows with null vlues
#df_nulls = clean_city_data[clean_city_data.isna().any(axis=1)]
#print(df_nulls)

# Display sample data
clean_city_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vardo,70.3705,31.1107,0.99,86,75,8.75,NO,1675177596
1,1,bluff,-46.6000,168.3333,15.43,97,100,2.59,NZ,1675177569
2,2,hobyo,5.3505,48.5268,25.26,71,2,9.23,SO,1675177569
3,3,lumsden,50.6501,-104.8678,-17.77,95,100,1.23,CA,1675177570
4,4,guerrero negro,27.9769,-114.0611,12.88,80,8,3.35,MX,1675177380


### Step 3: Create a new DataFrame called `hotel_df`.

In [45]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_data[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,vardo,NO,70.3705,31.1107,86,
1,bluff,NZ,-46.6000,168.3333,97,
2,hobyo,SO,5.3505,48.5268,71,
3,lumsden,CA,50.6501,-104.8678,95,
4,guerrero negro,MX,27.9769,-114.0611,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = 'circle:' + str(lon) + ',' + str(lat) + ',' + str(radius)
    params["bias"] = 'proximity:' + str(lon) + ',' + str(lat)
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vardo - nearest hotel: Meieriet
bluff - nearest hotel: No hotel found
hobyo - nearest hotel: No hotel found
lumsden - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
kapaa - nearest hotel: Pono Kai Resort
hithadhoo - nearest hotel: Pebbles Inn
port alfred - nearest hotel: No hotel found
buta - nearest hotel: Hôtel Rubi
sioux lookout - nearest hotel: Wellington Inn & Restaurant
jining - nearest hotel: No hotel found


KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [40]:
#%%capture --no-display

#Cleaning up hotel_df to remove rows with no hotel found
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != 'No hotel found']
clean_hotel_df.head()

# Configure the map plot
map_plot_1 = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City',
    alpha = .5,
    hover_cols = ['Country','Hotel Name']   
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)